In [17]:
using LinearAlgebra
using Plots
using Ripserer
using Distances
using Plots
using DataFrames, CSV
using DelimitedFiles
using AbstractAlgebra
using MultivariateStats #for pca
using IterativeSolvers
using NearestNeighbors
include("mv_function_bens_alg.jl")


mv (generic function with 1 method)

In [18]:
#Globals
K = 10 # in KNN
OG_K = 10
ITER = 5    # iterations of algorithm
min_loss = 1000
input_file ="./cool_big_data.csv"
output_file ="./point_color"

a_ = []
b_ = []
c_ = []
d_ = []
loss_ = []

Any[]

In [19]:
# Loss function 0.5 instead of pi

function loss(points, coord_ab, coord_cd)
    a = mod(coord_cd[points[1]],1.0)
    b = mod(coord_cd[points[2]],1.0)
    c = mod(coord_ab[points[3]],1.0)
    d = mod(coord_ab[points[4]],1.0)
    loss = (0.5 - mod(a-b, 1.0))^2 +  (0.5 - mod(c-d,1.0))^2
    return loss
end

loss (generic function with 1 method)

In [20]:
# Load Data
matrix = readdlm(input_file, ',', Float64)
data = []
for row in 1:length(matrix[:,1])
    arr = [matrix[row,i] for i in 1:length(matrix[1,:])]
    push!(data, arr)
end

# Do zeroth iteration of algerithm with random points

# Random distinct start points,
# Could be issues with distinct points yielding same piercing hole, and therefore cant do MV
a_0,b_0,c_0,d_0 = rand(1:25), rand(25:50), rand(50:75), rand(75:100)


# Store to see progress 

push!(a_, a_0)
push!(b_, b_0)
push!(c_, c_0)
push!(d_, d_0)



1-element Vector{Any}:
 98

In [21]:

for k in 1:ITER
    points = [a_[length(a_)], b_[length(b_)], c_[length(c_)], d_[length(d_)]]

    mv(points = points, start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
    matrix = readdlm(output_file * ".csv", ',')
    # indexing from 2 because c
    coord_ab = matrix[2:end,4]
    println(coord_ab[1])
    matrix = readdlm(output_file * "double" * ".csv", ',')
    coord_cd = matrix[2:end,4]

    a_0 = points[1]
    b_0 = points[2]
    d_0 = points[4]
    c_0 = points[3]

    
    vec_0 = Array([data[b_0][1],data[b_0][2],data[b_0][3]])
    vec_1 = Array([data[d_0][1],data[d_0][2],data[d_0][3]])
    
    data_matrix = zeros(Float64, (3, length(data)))

    for col in 1:length(data)
        data_matrix[:,col] = [data[col][1],data[col][2],data[col][3]]
    end

    kdtree = KDTree(data_matrix)
    b_0_set, dists = knn(kdtree, vec_0,K, true)
    d_0_set, dists = knn(kdtree, vec_1,K, true)


    # find b_0^i and d_0^i that minimizes loss

    b_0_i = b_0
    d_0_i = d_0

    # See if stationary
    old_b = b_0
    old_d = d_0

    for i in 1:length(b_0_set)
        b_0_i = b_0_set[i]
        for j in 1:length(d_0_set)
            d_0_i  = d_0_set[j]
            if loss([a_0,b_0_i,c_0,d_0_i], coord_ab, coord_cd) <= min_loss
                min_loss = loss([a_0,b_0_i,c_0,d_0_i], coord_ab, coord_cd)
                b_0 = b_0_i
                # Let's freeze one variable
                #d_0 = d_0_i 
            end
        end
    end
    
    # Repeat same thing but the other variable
    for i in 1:length(b_0_set)
        b_0_i = b_0_set[i]
        for j in 1:length(d_0_set)
            d_0_i  = d_0_set[j]
            if loss([a_0,b_0_i,c_0,d_0_i], coord_ab, coord_cd) <= min_loss
                min_loss = loss([a_0,b_0_i,c_0,d_0_i], coord_ab, coord_cd)
                #b_0 = b_0_i
                # Let's freeze one variable
                d_0 = d_0_i 
            end
        end
    end


    if old_b == b_0 && old_d == d_0

        println("THIS IS A FIXPOINT")
        break
        #K += 10
        #println("K = ", K)
    else
        K = OG_K
    end
    println("")
    println("Loss: ", min_loss)
    #println("Points vector: ", points)
    #println("Points: ", a_0,"-", b_0,"-", c_0,"-", d_0)
    println("")
    push!(a_, a_0)
    push!(b_, b_0)
    push!(c_, c_0)
    push!(d_, d_0)

    println("Done with: ", k)

end

Number of points: 100 


Careful if this number is too high: 4476


BoundsError: BoundsError: attempt to access Int64 at index [2]

In [22]:
print("[" * string(a_[1]-1) * "," * string(b_[1]-1) * "," * string(c_[1]-1) * "," * string(d_[1]-1) * "]")

[12,38,51,97]

In [23]:
l = length(a_)

print("[" * string(a_[l]-1) * "," * string(b_[l]-1) * "," * string(c_[l]-1) * "," * string(d_[l]-1) * "]")

[12,38,51,97]